# Test the two-line CLEDB inversion on synthetic CLE data.

In [1]:
## the script version "test_2line.py" is generated by exporting this notebook via file -> export as -> executable script.
## Do not directly edit that version!

<Figure size 1080x1080 with 0 Axes>

In [3]:
## Needed modules
import pickle                ## CLE synthetic observation data cube is saved in the pickle format.
import importlib             ## reloads imports (python default settings will not reload small changes).

from numba.typed import List ## Most numba functions are loaded by the CLEDB modules; non-reflected lists are needed to create a list of 2 bservation arrays

## Inversion Constants and control parameter imports.
## symbolic links pointing to the root directory containing the files are used. Pythondoes not allow relative upstream references anymore
import constants as consts
import ctrlparams 
params=ctrlparams.ctrlparams()    ## just a shorter label

### 1. Import the synthetic CLE observation.

In [4]:
## observations of a 3 dipole coronal structure of a Fe XIII combined observation
## sobs1-3 are the independent dipoles
## sobsa is the combined 3 dipole output
## waveA and waveB are the wavelength arrays for the two Fe XIII lines

with open('obsstokes_3dipole_hires_fullspectra.pkl','rb') as f:
    sobs1,sobs2,sobs3,sobsa,waveA,waveB = pickle.load(f)  
### reversing of the wavelength range. THIS IS NEEDED! CLE writes frequency-wise, so wavelengths are reversed in the original datacubes!!!!!!
sobs1=sobs1[:,:,::-1,:]   
sobs2=sobs2[:,:,::-1,:]
sobs3=sobs3[:,:,::-1,:]
sobsa=sobsa[:,:,::-1,:]
waveA=waveA[::-1]
waveB=waveB[::-1]
## we test here using sobs1.

In [4]:
## load the fake observation muram data.
## FE XIII 1074+1079
## sobs_a is normalized to first column (stokes I 1074)
## RMS is 1 in this example; observation can be convoluted with noise if needed and re-normalized again

# with open('obsstokes_avg_muram.pkl','rb') as f:
#     f1aa,f2aa,sobs_a,yobs_a,rms,wvl,xxl,yyl = pickle.load(f)    

# with open('obsstokes_avg_muram2.pkl','rb') as f:
#     sobsa,yobsa,rms,xxl,yyl = pickle.load(f)    


### 2. Test the CLEDB_PREPINV module with synthetic data. 

##### Remember to set your personal options and database paths in the ctrlparams class (in the parent directory) before continuing.

In [5]:
import CLEDB_PREPINV.CLEDB_PREPINV as prepinv  ##imports from the CLEDB_PREPINV subdirectory

In [5]:
## arrange the two observation "files" in a simple list;
## un-necesary step given the shape of sobs array, but it mimicks a file/header structure.

## set a proper numba typed list
sobs_lst=[sobsa[:,:,:,0:4],sobsa[:,:,:,4:8]]  ## standard python lists will be deprecated ; they do not work well with numba
sobs_in = List()                              ## this is the List object implemented by numba; It utilizes memory in a column-like fashion.
[sobs_in.append(x) for x in sobs_lst]         ## Numba developers claim that it is a significantly faster performing object

(600, 280, 30, 8)


##### preprocess the observation "files"

In [7]:
# importlib.reload(prepinv)       ## If module is modified, reload the contents

sobs_tot,yobs,rms,background,keyvals,sobs_totrot,aobs=prepinv.sobs_preprocess(sobs_in,params)

In [9]:
## select and pre-read the database files based on the observation preprocessing
## At this point, the database should be generated and properly linked.

importlib.reload(prepinv)       ## If module is modified, reload the contents
db_enc,database,dbhdr=prepinv.sdb_preprocess(yobs,keyvals,params)
## Warning, this might be memory consuming; see details in documentation

##### At this point all necesary data and required databases are loaded into memory for fast processing

### 3. Test the CLEDB_PROC module with the same synthetic data.

In [10]:
import CLEDB_PROC.CLEDB_PROC as procinv

##### Process the spectroscopy outputs

In [11]:
#importlib.reload(procinv)       ## If module is modified, reload the contents 
specout=procinv.spectro_proc(sobs_in,sobs_tot,rms,background,keyvals,consts,params)

##### Process the full vector magnetic field inversion products

In [ ]:
# importlib.reload(procinv)       ## If module is modified, reload the contents
invout,sfound=procinv.cledb_invproc(sobs_totrot,database,db_enc,yobs,aobs,rms,dbhdr,keyvals,params.nsearch,params.maxchisq,params.bcalc,params.reduced,params.verbose)
## WARNING: This step has a significantly long execution time.

##### All should be good if we reached this point; all the outputs should be computed.

### 4. DUMP results (optional)

In [8]:
from datetime import datetime
datestamp = datetime.now().strftime("%Y%m%d-%H:%M:%S")

with open(f'outparams_2line_{datestamp}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([specout,invout,sfound], f)

### 5. PLOT the outputs (optional)

In [10]:
##needed libraries and functions
from matplotlib import pyplot as plt
#%matplotlib widget                ## interactive plotting; use only on local machines if installed

# colorbar function to have nice colorbars in figures
def colorbar(mappable):
   from mpl_toolkits.axes_grid1 import make_axes_locatable
   import matplotlib.pyplot as plt
   last_axes = plt.gca()
   ax = mappable.axes
   fig = ax.figure
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="3.5%", pad=0.05)
   cbar = fig.colorbar(mappable, cax=cax)
   #cbar.formatter.set_powerlimits((0,4))
   plt.sca(last_axes)
   return cbar

In [21]:
## Plot spectroscopy

linen=0                ## choose which line to plot; range is [0:1] for single or double line input

fig, plots = plt.subplots(nrows=3, ncols=4, figsize=(12,8))
ab=plots[0,0].imshow(specout[230:400,65:195,linen,0],vmin=1074.45,vmax=1074.72)
plots[0,0].set_title('Wavelength')
colorbar(ab)

ab=plots[0,1].imshow(specout[230:400,65:195,linen,1],vmin=-0.5,vmax=0.5)
plots[0,1].set_title('Doppler shift [nm]')
colorbar(ab)

ab=plots[0,2].imshow(specout[230:400,65:195,linen,2],vmin=-40,vmax=40)
plots[0,2].set_title('Doppler shift [km/s]')
colorbar(ab)

ab=plots[0,3].imshow(specout[230:400,65:195,linen,7],vmin=0,vmax=0.01)
plots[0,3].set_title('Background Stokes I')
colorbar(ab)


ab=plots[1,0].imshow(specout[230:400,65:195,linen,3],vmin=0,vmax=5)
plots[1,0].set_title('Stokes I int. (linecore)')
colorbar(ab)

ab=plots[1,1].imshow(specout[230:400,65:195,linen,4],vmin=-0.05,vmax=0.05)
plots[1,1].set_title('Stokes Q int. (linecore)')
colorbar(ab)

ab=plots[1,2].imshow(specout[230:400,65:195,linen,5],vmin=-0.05,vmax=0.05)
plots[1,2].set_title('Stokes U int. (linecore)')
colorbar(ab)

ab=plots[1,3].imshow(specout[230:400,65:195,linen,6],vmin=-0.08,vmax=0.08)
plots[1,3].set_title('Stokes V int. (linecore)')
colorbar(ab)



ab=plots[2,0].imshow(specout[230:400,65:195,linen,8],vmin=0.05,vmax=0.2)
plots[2,0].set_title('Line FWHM')
colorbar(ab)

ab=plots[2,1].imshow(specout[230:400,65:195,linen,9],vmin=0.05,vmax=0.2)
plots[2,1].set_title('Non-thermal width')
colorbar(ab)

ab=plots[2,2].imshow(specout[230:400,65:195,linen,10],vmin=0.0,vmax=0.3)
plots[2,2].set_title('Linear polarization fraction')
colorbar(ab)

ab=plots[2,3].imshow(specout[230:400,65:195,linen,11],vmin=0.0,vmax=0.3)
plots[2,3].set_title('Total polarization fraction')
colorbar(ab)
plt.tight_layout()
plt.savefig(f"specout_2line_{datestamp}.pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
## Plot magnetic inversion

soln=3 ## which solution to plot; range is [0:nsearch-1] where nsearch is defined in ctrlparams

fig, plots = plt.subplots(nrows=4, ncols=3, figsize=(10,12))
ab=plots[0,0].imshow(invout[230:400,65:195,soln,0],vmin=0,vmax=5e7)
plots[0,0].set_title('Database index')
colorbar(ab)

ab=plots[0,1].imshow(invout[230:400,65:195,soln,1],vmin=0,vmax=0.25)
plots[0,1].set_title('Fit $\chi^2$')
colorbar(ab)

plots[0, 2].axis('off') ###leave one empty panel

ab=plots[1,0].imshow(invout[230:400,65:195,soln,2],vmin=1,vmax=10)
plots[1,0].set_title('Plasma density (Log)')
colorbar(ab)

ab=plots[1,1].imshow(invout[230:400,65:195,soln,3],vmin=0.9,vmax=1.5)
plots[1,1].set_title('Computed height (R$_\odot$)')
colorbar(ab)

ab=plots[1,2].imshow(invout[230:400,65:195,soln,4],vmin=-1.5,vmax=1.5)
plots[1,2].set_title('LOS position (R$_\odot$')
colorbar(ab)


ab=plots[2,0].imshow(invout[230:400,65:195,soln,5],vmin=0,vmax=700)
plots[2,0].set_title('LVS field strength')
colorbar(ab)

ab=plots[2,1].imshow(invout[230:400,65:195,soln,6],vmin=0,vmax=6.28)
plots[2,1].set_title('LVS B$_\phi$ [rad]')
colorbar(ab)

ab=plots[2,2].imshow(invout[230:400,65:195,soln,7],vmin=0,vmax=3.14)
plots[2,2].set_title('LVS B$_\Theta$ [rad]')
colorbar(ab)


ab=plots[3,0].imshow(invout[230:400,65:195,soln,8],vmin=-500,vmax=500)
plots[3,0].set_title('LOS cartesian B$_x$')
colorbar(ab)

ab=plots[3,1].imshow(invout[230:400,65:195,soln,9],vmin=-400,vmax=400)
plots[3,1].set_title('LOS cartesian B$_y$')
colorbar(ab)

ab=plots[3,2].imshow(invout[230:400,65:195,soln,10],vmin=-400,vmax=400)
plots[3,2].set_title('LOS cartesian B$_z$')
colorbar(ab)
plt.tight_layout()
plt.savefig(f"invout_2line_{datestamp}.pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
##Print inversion solution in a human readable way

# np.set_printoptions(linewidth=200,suppress=False)   ## Suppress can be set to true to disable exponential notation.
# xx=310      ## x pixel position
# yy=105      ## y pixel positions


# print("||    DB Index   ||     chi^2    ||  ne density  ||  y (height)  || x (LOS pos.) ||      B       ||    B_theta   ||    B_phi     ||      Bx      ||      By      ||     Bz       ||")
# print(invout[xx,yy,:,:])


||    DB Index   ||     chi^2    ||  ne density  ||  y (height)  || x (LOS pos.) ||      B       ||    B_theta   ||    B_phi     ||      Bx      ||      By      ||     Bz       ||
[[ 6.00269300e+06  5.32047396e-03  8.30495161e+00  1.06279408e+00 -9.98135587e-01  6.82639447e+02  3.36975285e+00  1.87083627e+00 -6.35241512e+02 -1.47505322e+02 -2.01759798e+02]
 [ 6.64959700e+06  5.32047396e-03  8.24417754e+00  1.06279408e+00  1.04932203e+00  6.82639447e+02  2.94853375e+00  1.30605551e+00 -6.46616252e+02  1.26409444e+02  1.78618841e+02]
 [ 6.00269200e+06  5.32048424e-03  8.30495161e+00  1.06279408e+00 -9.98135587e-01  6.76194813e+02  3.36975285e+00  1.83553747e+00 -6.35722759e+02 -1.47617069e+02 -1.76932756e+02]
 [ 6.64959800e+06  5.32048424e-03  8.24417754e+00  1.06279408e+00  1.04932203e+00  6.76194813e+02  2.94853375e+00  1.34135431e+00 -6.46240909e+02  1.26336067e+02  1.53789828e+02]]
